# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# Verify current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Uncomment the code below if you would like to get total number of rows 
print("Number of rows:", len(full_data_rows_list))
# Uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Number of rows: 8056


In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of rows in csv:", sum(1 for line in f))

Number of rows in csv: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Select the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

## Create table
query_create = 'CREATE TABLE IF NOT EXISTS song_session_item'
query_create = query_create + '(session_id int, item_in_session int, song_artist text, song_title text, song_length decimal, PRIMARY KEY(session_id, item_in_session))'

try:
    session.execute(query_create)
except Exception as e:
    print(e)
    
# Create SELECT statement for query #1
query_select = "select song_artist, song_title, song_length from song_session_item  WHERE session_id = 338 and item_in_session = 4"

# Run query to verify table was created
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
# Setup counter for counting rows
count = 0

# Loop through query results
for row in rows:
    count += 1
    print (row.song_artist, row.song_title, row.song_length)
  
# Print number of rows (should be 0)
print("Rows:", count)           
             

Faithless Music Matters (Mark Knight Dub) 495.3073
Rows: 1


In [9]:
# Set file to process
file = 'event_datafile_new.csv'

# Loop through file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Create insert query
        query_insert = "INSERT INTO song_session_item(session_id, item_in_session, song_artist, song_title, song_length)"
        query_insert = query_insert + "VALUES(%s, %s, %s, %s, %s)"
        # Assign appropriate column elements for each column in the INSERT statement.
        session.execute(query_insert, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))           
            

In [10]:
# Run query #1 again
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.song_artist, row.song_title, row.song_length])

# Print results    
print(t)       
    

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# Query 2: Select the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

# Create table
query_create2 = 'CREATE TABLE IF NOT EXISTS song_user_session'
query_create2 = query_create2 + '(user_id int, session_id int, song_artist text, song_title text, item_in_session int, first_name text, last_name text, PRIMARY KEY(user_id, session_id, item_in_session))'

try:
    session.execute(query_create2)
except Exception as e:
    print(e)

# Loop through file    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Create insert query
        query_insert2 = "INSERT INTO song_user_session(user_id, session_id, song_artist, song_title, item_in_session, first_name, last_name)"
        query_insert2 = query_insert2 + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign appropriate column elements for each column in the INSERT statement.
        session.execute(query_insert2, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4])) 

# Create SELECT statement for query #2        
query_select2 = "select song_artist, song_title, first_name, last_name from song_user_session WHERE user_id = 10 and session_id = 182"

# Run query #2
try:
    rows2 = session.execute(query_select2)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])
for row in rows2:
    t.add_row([row.song_artist, row.song_title, row.first_name, row.last_name])

# Print results    
print(t)    

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [12]:
# Query 3: Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create table
query_create3 = 'CREATE TABLE IF NOT EXISTS from_song_title'
query_create3 = query_create3 + '(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id))'

try:
    session.execute(query_create3)
except Exception as e:
    print(e)
    
# Loop through file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Create insert query
        query_insert3 = "INSERT INTO from_song_title(song_title, user_id, first_name, last_name)"
        query_insert3 = query_insert3 + "VALUES(%s, %s, %s, %s)"
        
        ## Assign appropriate column elements for each column in the INSERT statement.
        session.execute(query_insert3, (line[9], int(line[10]), line[1], line[4])) 

# Create SELECT statement for query #3
query_select3 = "select first_name, last_name from from_song_title WHERE song_title = 'All Hands Against His Own'"

# Run query #3
try:
    rows3 = session.execute(query_select3)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['First Name', 'Last Name'])
for row in rows3:
    t.add_row([row.first_name, row.last_name])

# Print results    
print(t)                       

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [13]:
# Drop the tables before closing out the sessions

In [14]:
query_drop = "drop table song_session_item"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)
    
query_drop2 = "drop table song_user_session"
try:
    rows = session.execute(query_drop2)
except Exception as e:
    print(e)
    
query_drop3 = "drop table from_song_title"
try:
    rows = session.execute(query_drop3)
except Exception as e:
    print(e) 
    

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()